In [1]:
# importing libraries for data modeling

import pandas as pd
import numpy as np
import collections
from nltk.tokenize.treebank import TreebankWordTokenizer
from nltk.tokenize import TweetTokenizer
from common import utils, vocabulary
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import preprocessing
import tensorflow as tf
import os
import time
import datetime
from CNN.text_cnn import TextCNN
from tensorflow.contrib import learn
import sys

/home/chadharness/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [1]:
!pwd

/home/chadharness/w210/Parlancr/models/classifier


In [2]:
# loading model data 

don_train = pd.read_csv("../../../language-style-transfer-test-v1/language-style-transfer/data/yelp/reddit_thedonald.train.0", delimiter="\t", header = None, names = ['sentence_text'])
teen_train = pd.read_csv("../../../language-style-transfer-test-v1/language-style-transfer/data/yelp/reddit_thedonald.train.1", delimiter="\t", header = None, names = ['sentence_text'])
don_test = pd.read_csv("../../../language-style-transfer-test-v1/language-style-transfer/data/yelp/reddit_thedonald.test.0", delimiter="\t", header = None, names = ['sentence_text'])
teen_test = pd.read_csv("../../../language-style-transfer-test-v1/language-style-transfer/data/yelp/reddit_thedonald.test.1", delimiter="\t", header = None, names = ['sentence_text'])

don_train['group']='thedonald'
don_test['group']='thedonald'
teen_train['group']='teenagers'
teen_test['group']='teenagers'

#combine dem/rep data together for training/testing
#shuffle it for random order
all_train = don_train.append(teen_train)
all_test = don_test.append(teen_test)

all_train=all_train.sample(frac=.5)
all_test=all_test.sample(frac=1)

print(all_train.head())
print(all_test.head())

                                            sentence_text      group
7264                                    i just found this  thedonald
9045    https m imgur com a dofdz and it s not too bad...  teenagers
108767  damn i know about just about everything but ne...  thedonald
197980  im glad i can find people like you that actual...  teenagers
62916   pssh gowdy is going after DG fbi janitors and ...  thedonald
                                           sentence_text      group
4849                                                  oh  teenagers
18518  that might explain the theories why he was kil...  thedonald
50929                               same but minus the s  teenagers
57137                               seduce him obviously  teenagers
15188  who are you and what did you do to the real u ...  teenagers


In [3]:
print(all_train.shape)

(220876, 2)


In [5]:
smpl_train = all_train.sample(n=100000)
smpl_test = all_test.sample(n=100000)
print(smpl_train.shape)
print(smpl_test.shape)

(100000, 2)
(100000, 2)


In [6]:
train_x = smpl_train['sentence_text']
train_y = smpl_train['group']
test_x = smpl_test['sentence_text']
test_y = smpl_test['group']

In [7]:
# tokenize data in comment body

tokenizer = TweetTokenizer()

train_x_tokens = []
test_x_tokens = []

for x in train_x:
    train_x_tokens.append(tokenizer.tokenize(x))
    
for x in test_x:
    test_x_tokens.append(tokenizer.tokenize(x))

In [8]:
# define vocabulary using w266 common vocab

train_text_all = [item for sublist in train_x_tokens for item in sublist]

vocab = vocabulary.Vocabulary(train_text_all, size=20000)
print("Vocabulary size: {:,}".format(vocab.size))
print("Vocabulary dict: ", vocab.word_to_id)

train_x_ids = []
test_x_ids = []

for x in train_x_tokens:
    train_x_ids.append(vocab.words_to_ids(x))
    
for x in test_x_tokens:
    test_x_ids.append(vocab.words_to_ids(x))

Vocabulary size: 20,000
Vocabulary dict:  {'<s>': 0, '</s>': 1, '<unk>': 2, 'i': 3, 'the': 4, 'a': 5, 'to': 6, 'you': 7, 'it': 8, 'and': 9, 'is': 10, 'that': 11, 's': 12, 'of': 13, 't': 14, 'in': 15, 'this': 16, 'for': 17, 'n': 18, 'do': 19, 'my': 20, 'not': 21, 'are': 22, 'they': 23, 'but': 24, 'have': 25, 'be': 26, 'me': 27, 'was': 28, 'like': 29, 'on': 30, 'DG': 31, 'what': 32, 'no': 33, 'm': 34, 'just': 35, 'he': 36, 'so': 37, 'with': 38, 'if': 39, 'we': 40, 'your': 41, 'all': 42, 'DGDG': 43, 'or': 44, 'good': 45, 'at': 46, 'can': 47, 're': 48, 'as': 49, 'about': 50, 'get': 51, 'u': 52, 'people': 53, 'one': 54, 'how': 55, 'up': 56, 'out': 57, 'there': 58, 'would': 59, 'from': 60, 'know': 61, 'too': 62, 'will': 63, 'did': 64, 'them': 65, 'https': 66, 'why': 67, 'she': 68, 'think': 69, 'com': 70, 'an': 71, 'by': 72, 'now': 73, 'really': 74, 'more': 75, 'shit': 76, 'who': 77, 'yes': 78, 'when': 79, 'yeah': 80, 'some': 81, 'because': 82, 'their': 83, 'has': 84, 'here': 85, 'got': 86, '

In [9]:
# count token occurences and convert to feature vector for BoW model

train_x_fdict = []
test_x_fdict = []

for x in train_x_ids:
    train_x_fdict.append(collections.Counter(x))
    
for x in test_x_ids:
    test_x_fdict.append(collections.Counter(x))

train_x_vector = []
test_x_vector = []

num_features = vocab.size
print('creating id lists...')
for x in train_x_fdict:
    train_x_vector.append([x.get(i, 0) for i in range(num_features)])
    
for x in test_x_fdict:
    test_x_vector.append([x.get(i, 0) for i in range(num_features)])

# use w266 common utils to convert id lists to sparse bow matrix
print('starting id lists to sparse bow conversion...')
train_x_sparse_bow = utils.id_lists_to_sparse_bow(train_x_fdict, vocab.size)
test_x_sparse_bow = utils.id_lists_to_sparse_bow(test_x_fdict, vocab.size)

# training Multinomial Naive Bayes for simple baseline model
print('training Multinomial Naive Bayes for simple baseline model...')
nb = MultinomialNB()
nb.fit(train_x_sparse_bow, train_y)
y_pred = nb.predict(test_x_sparse_bow)

acc = accuracy_score(test_y, y_pred)
print("Accuracy on test set: {:.02%}".format(acc))
classrep = classification_report(test_y, y_pred)
print(classrep)

creating id lists...
starting id lists to sparse bow conversion...
training Multinomial Naive Bayes for simple baseline model...
Accuracy on test set: 80.05%
             precision    recall  f1-score   support

  teenagers       0.81      0.88      0.84     60313
  thedonald       0.79      0.68      0.73     39687

avg / total       0.80      0.80      0.80    100000



In [11]:
# pad all id vectors to max length for Neural model

max_comment_length = max([len(x) for x in train_x_ids])

train_x_neural = train_x_ids
test_x_neural = test_x_ids

for x in train_x_neural:
    if len(x) < max_comment_length:
        for n in range(max_comment_length - len(x)):
            x.append(1)
    elif len(x) > max_comment_length:
        x = x[:max_comment_length]
        
for x in test_x_neural:
    if len(x) < max_comment_length:
        for n in range(max_comment_length - len(x)):
            x.append(1)
    elif len(x) > max_comment_length:
        x = x[:max_comment_length]

# convert labels to one-hot vectors for Neural model --update your class lables below!!

terms_list = [['thedonald'],['teenagers']]
lb = preprocessing.MultiLabelBinarizer()
lb.fit(terms_list)

train_labels = np.array(train_y.tolist())
test_labels = np.array(test_y.tolist())

train_lables_format = [train_labels[i:i+1] for i in range(0, len(train_labels), 1)]    
test_lables_format = [test_labels[i:i+1] for i in range(0, len(test_labels), 1)]

train_y_neural = lb.transform(train_lables_format)
test_y_neural = lb.transform(test_lables_format)

##### The best run:
Evaluation:
2018-08-05T21:08:26.820785: step 6100, loss 0.44334, acc 0.80118

Saved model checkpoint to /home/chadharness/w210/Parlancr/models/classifier/runs/1533502250/checkpoints/model-6100

In [16]:
# creating and training CNN for deep classification
# CNN model structure and code based on an implementation of Kim Yoon's "Convolutional Neural Networks for Sentence Classification"
# From https://github.com/dennybritz/cnn-text-classification-tf

# Parameters
# ==================================================

# Model Hyperparameters
tf.flags.DEFINE_integer("embedding_dim", 128, "Dimensionality of character embedding (default: 128)")
tf.flags.DEFINE_string("filter_sizes", "3,4,5", "Comma-separated filter sizes (default: '3,4,5')")
tf.flags.DEFINE_integer("num_filters", 128, "Number of filters per filter size (default: 128)")
tf.flags.DEFINE_float("dropout_keep_prob", 0.5, "Dropout keep probability (default: 0.5)")
tf.flags.DEFINE_float("l2_reg_lambda", 0.0, "L2 regularization lambda (default: 0.0)")

# Training parameters
tf.flags.DEFINE_integer("batch_size", 64, "Batch Size (default: 64)")
tf.flags.DEFINE_integer("num_epochs", 5, "Number of training epochs (default: 200)")
tf.flags.DEFINE_integer("evaluate_every", 100, "Evaluate model on dev set after this many steps (default: 100)")
tf.flags.DEFINE_integer("checkpoint_every", 100, "Save model after this many steps (default: 100)")
tf.flags.DEFINE_integer("num_checkpoints", 7, "Number of checkpoints to store (default: 5)")

# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")
tf.app.flags.DEFINE_string('f', '', 'kernel')

FLAGS = tf.flags.FLAGS
FLAGS(sys.argv)
print("\nParameters:")
for attr, value in sorted(FLAGS.__flags.items()):
    print("{}={}".format(attr.upper(), value))
print("")


# Data Preparation
# ==================================================

x_train = np.array(train_x_neural)
y_train = np.array(train_y_neural)
x_dev = np.array(test_x_neural)
y_dev = np.array(test_y_neural)

print("x_train", np.shape(x_train))
print("y_train", np.shape(y_train))
print("x_dev", np.shape(x_dev))
print("x_dev", np.shape(y_dev))

print("Vocabulary Size: {:d}".format(vocab.size))
print("Train/Dev split: {:d}/{:d}".format(len(y_train), len(y_dev)))


# Training
# ==================================================

def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]
            
with tf.Graph().as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        cnn = TextCNN(
            sequence_length=x_train.shape[1],
            num_classes=2,
            vocab_size=vocab.size,
            embedding_size=FLAGS.embedding_dim,
            filter_sizes=list(map(int, FLAGS.filter_sizes.split(","))),
            num_filters=FLAGS.num_filters,
            l2_reg_lambda=FLAGS.l2_reg_lambda)

        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(1e-3)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

        # Keep track of gradient values and sparsity (optional)
        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.summary.merge(grad_summaries)

        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", cnn.loss)
        acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)

        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)

        # Initialize all variables
        sess.run(tf.global_variables_initializer())

        def train_step(x_batch, y_batch):
            """
            A single training step
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: FLAGS.dropout_keep_prob
            }
            _, step, summaries, loss, accuracy = sess.run(
                [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            train_summary_writer.add_summary(summaries, step)

        def dev_step(x_batch, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: 1.0
            }
            step, summaries, loss, accuracy = sess.run(
                [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            if writer:
                writer.add_summary(summaries, step)

        # Generate batches
        batches = batch_iter(
            list(zip(x_train, y_train)), FLAGS.batch_size, FLAGS.num_epochs)
        # Training loop. For each batch...
        for batch in batches:
            x_batch, y_batch = zip(*batch)
            train_step(x_batch, y_batch)
            current_step = tf.train.global_step(sess, global_step)
            if current_step % FLAGS.evaluate_every == 0:
                print("\nEvaluation:")
                dev_step(x_dev, y_dev, writer=dev_summary_writer)
                print("")
            if current_step % FLAGS.checkpoint_every == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))


Parameters:
ALLOW_SOFT_PLACEMENT=<absl.flags._flag.BooleanFlag object at 0x7f07c55faef0>
BATCH_SIZE=<absl.flags._flag.Flag object at 0x7f07c55faac8>
CHECKPOINT_EVERY=<absl.flags._flag.Flag object at 0x7f07c55facc0>
DROPOUT_KEEP_PROB=<absl.flags._flag.Flag object at 0x7f07c55fa9e8>
EMBEDDING_DIM=<absl.flags._flag.Flag object at 0x7f07c55fa898>
EVALUATE_EVERY=<absl.flags._flag.Flag object at 0x7f07c55fac18>
F=<absl.flags._flag.Flag object at 0x7f07c55ef048>
FILTER_SIZES=<absl.flags._flag.Flag object at 0x7f07c55fa828>
L2_REG_LAMBDA=<absl.flags._flag.Flag object at 0x7f07c55faa20>
LOG_DEVICE_PLACEMENT=<absl.flags._flag.BooleanFlag object at 0x7f07c55faf98>
NUM_CHECKPOINTS=<absl.flags._flag.Flag object at 0x7f07c55fad68>
NUM_EPOCHS=<absl.flags._flag.Flag object at 0x7f07c55fab70>
NUM_FILTERS=<absl.flags._flag.Flag object at 0x7f07c55fa908>

x_train (100000, 24)
y_train (100000, 2)
x_dev (100000, 24)
x_dev (100000, 2)
Vocabulary Size: 20000
Train/Dev split: 100000/100000
INFO:tensorflow:Su

2018-08-05T20:50:58.195689: step 85, loss 1.31507, acc 0.53125
2018-08-05T20:50:58.278446: step 86, loss 1.25605, acc 0.5625
2018-08-05T20:50:58.358405: step 87, loss 1.31186, acc 0.5
2018-08-05T20:50:58.437801: step 88, loss 1.11419, acc 0.578125
2018-08-05T20:50:58.512698: step 89, loss 1.04345, acc 0.625
2018-08-05T20:50:58.593907: step 90, loss 0.83904, acc 0.65625
2018-08-05T20:50:58.678279: step 91, loss 1.42653, acc 0.484375
2018-08-05T20:50:58.753985: step 92, loss 1.06595, acc 0.609375
2018-08-05T20:50:58.833499: step 93, loss 1.31733, acc 0.46875
2018-08-05T20:50:58.910862: step 94, loss 0.986727, acc 0.625
2018-08-05T20:50:58.996069: step 95, loss 0.902608, acc 0.640625
2018-08-05T20:50:59.082459: step 96, loss 1.01861, acc 0.625
2018-08-05T20:50:59.160640: step 97, loss 0.964208, acc 0.671875
2018-08-05T20:50:59.241574: step 98, loss 0.869383, acc 0.65625
2018-08-05T20:50:59.323212: step 99, loss 1.11676, acc 0.546875
2018-08-05T20:50:59.405501: step 100, loss 1.52596, acc 

2018-08-05T20:51:24.614564: step 208, loss 0.767037, acc 0.703125
2018-08-05T20:51:24.706300: step 209, loss 0.86722, acc 0.625
2018-08-05T20:51:24.796716: step 210, loss 0.829165, acc 0.65625
2018-08-05T20:51:24.889595: step 211, loss 0.633514, acc 0.640625
2018-08-05T20:51:24.978919: step 212, loss 0.872715, acc 0.609375
2018-08-05T20:51:25.071719: step 213, loss 0.896361, acc 0.578125
2018-08-05T20:51:25.171263: step 214, loss 0.551581, acc 0.78125
2018-08-05T20:51:25.261757: step 215, loss 0.823738, acc 0.59375
2018-08-05T20:51:25.358185: step 216, loss 0.935047, acc 0.578125
2018-08-05T20:51:25.447388: step 217, loss 0.769862, acc 0.640625
2018-08-05T20:51:25.538310: step 218, loss 1.05636, acc 0.515625
2018-08-05T20:51:25.634787: step 219, loss 0.96417, acc 0.59375
2018-08-05T20:51:25.732010: step 220, loss 0.762014, acc 0.640625
2018-08-05T20:51:25.827856: step 221, loss 0.773706, acc 0.625
2018-08-05T20:51:25.926211: step 222, loss 0.682314, acc 0.640625
2018-08-05T20:51:26.023

2018-08-05T20:51:45.478780: step 332, loss 0.760591, acc 0.609375
2018-08-05T20:51:45.569849: step 333, loss 0.772468, acc 0.625
2018-08-05T20:51:45.659982: step 334, loss 0.678806, acc 0.703125
2018-08-05T20:51:45.754999: step 335, loss 0.726474, acc 0.640625
2018-08-05T20:51:45.846029: step 336, loss 0.728877, acc 0.59375
2018-08-05T20:51:45.939987: step 337, loss 0.862671, acc 0.625
2018-08-05T20:51:46.033547: step 338, loss 0.56771, acc 0.765625
2018-08-05T20:51:46.127581: step 339, loss 0.796254, acc 0.640625
2018-08-05T20:51:46.217062: step 340, loss 0.618528, acc 0.625
2018-08-05T20:51:46.309847: step 341, loss 0.784229, acc 0.65625
2018-08-05T20:51:46.403415: step 342, loss 0.719524, acc 0.6875
2018-08-05T20:51:46.495386: step 343, loss 0.61198, acc 0.703125
2018-08-05T20:51:46.582267: step 344, loss 0.617916, acc 0.75
2018-08-05T20:51:46.675872: step 345, loss 0.684888, acc 0.609375
2018-08-05T20:51:46.773733: step 346, loss 0.645737, acc 0.703125
2018-08-05T20:51:46.870324: s

2018-08-05T20:52:05.025783: step 456, loss 0.681211, acc 0.65625
2018-08-05T20:52:05.120101: step 457, loss 0.728051, acc 0.546875
2018-08-05T20:52:05.212247: step 458, loss 0.500609, acc 0.75
2018-08-05T20:52:05.310861: step 459, loss 0.819085, acc 0.546875
2018-08-05T20:52:05.408939: step 460, loss 0.575158, acc 0.75
2018-08-05T20:52:05.491503: step 461, loss 0.673344, acc 0.703125
2018-08-05T20:52:05.581980: step 462, loss 0.722047, acc 0.59375
2018-08-05T20:52:05.668422: step 463, loss 0.665188, acc 0.6875
2018-08-05T20:52:05.761406: step 464, loss 0.588943, acc 0.6875
2018-08-05T20:52:05.855502: step 465, loss 0.632202, acc 0.640625
2018-08-05T20:52:05.939198: step 466, loss 0.585331, acc 0.734375
2018-08-05T20:52:06.032732: step 467, loss 0.576476, acc 0.640625
2018-08-05T20:52:06.130142: step 468, loss 0.627055, acc 0.6875
2018-08-05T20:52:06.220857: step 469, loss 0.665186, acc 0.609375
2018-08-05T20:52:06.313728: step 470, loss 0.655919, acc 0.59375
2018-08-05T20:52:06.406887:

...

2018-08-05T21:08:17.729840: step 6089, loss 0.521677, acc 0.78125
2018-08-05T21:08:17.822142: step 6090, loss 0.254022, acc 0.859375
2018-08-05T21:08:17.913319: step 6091, loss 0.376112, acc 0.84375
2018-08-05T21:08:18.009605: step 6092, loss 0.27048, acc 0.90625
2018-08-05T21:08:18.102048: step 6093, loss 0.291296, acc 0.875
2018-08-05T21:08:18.190601: step 6094, loss 0.44701, acc 0.75
2018-08-05T21:08:18.288221: step 6095, loss 0.299493, acc 0.90625
2018-08-05T21:08:18.382201: step 6096, loss 0.355127, acc 0.84375
2018-08-05T21:08:18.474664: step 6097, loss 0.423833, acc 0.78125
2018-08-05T21:08:18.564265: step 6098, loss 0.300836, acc 0.84375
2018-08-05T21:08:18.657962: step 6099, loss 0.426788, acc 0.828125
2018-08-05T21:08:18.748538: step 6100, loss 0.276311, acc 0.890625

Evaluation:
2018-08-05T21:08:26.820785: step 6100, loss 0.44334, acc 0.80118

Saved model checkpoint to /home/chadharness/w210/Parlancr/models/classifier/runs/1533502250/checkpoints/model-6100

2018-08-05T21:08:

2018-08-05T21:08:45.018979: step 6208, loss 0.362685, acc 0.859375
2018-08-05T21:08:45.112909: step 6209, loss 0.298443, acc 0.859375
2018-08-05T21:08:45.194922: step 6210, loss 0.46293, acc 0.78125
2018-08-05T21:08:45.294131: step 6211, loss 0.226029, acc 0.921875
2018-08-05T21:08:45.390193: step 6212, loss 0.296976, acc 0.921875
2018-08-05T21:08:45.483123: step 6213, loss 0.315628, acc 0.859375
2018-08-05T21:08:45.571804: step 6214, loss 0.288784, acc 0.859375
2018-08-05T21:08:45.667612: step 6215, loss 0.344174, acc 0.84375
2018-08-05T21:08:45.755401: step 6216, loss 0.374779, acc 0.875
2018-08-05T21:08:45.851294: step 6217, loss 0.426288, acc 0.859375
2018-08-05T21:08:45.942574: step 6218, loss 0.287096, acc 0.859375
2018-08-05T21:08:46.032715: step 6219, loss 0.274006, acc 0.90625
2018-08-05T21:08:46.129575: step 6220, loss 0.367284, acc 0.828125
2018-08-05T21:08:46.222862: step 6221, loss 0.458784, acc 0.78125
2018-08-05T21:08:46.311194: step 6222, loss 0.352318, acc 0.8125
2018-

2018-08-05T21:09:04.573716: step 6332, loss 0.305386, acc 0.90625
2018-08-05T21:09:04.665290: step 6333, loss 0.291507, acc 0.890625
2018-08-05T21:09:04.755297: step 6334, loss 0.216636, acc 0.890625
2018-08-05T21:09:04.843191: step 6335, loss 0.23172, acc 0.90625
2018-08-05T21:09:04.928322: step 6336, loss 0.319073, acc 0.859375
2018-08-05T21:09:05.016941: step 6337, loss 0.376758, acc 0.84375
2018-08-05T21:09:05.110383: step 6338, loss 0.238556, acc 0.9375
2018-08-05T21:09:05.199479: step 6339, loss 0.337528, acc 0.84375
2018-08-05T21:09:05.294429: step 6340, loss 0.252259, acc 0.875
2018-08-05T21:09:05.382627: step 6341, loss 0.412002, acc 0.8125
2018-08-05T21:09:05.472198: step 6342, loss 0.237825, acc 0.875
2018-08-05T21:09:05.567622: step 6343, loss 0.165725, acc 0.9375
2018-08-05T21:09:05.663468: step 6344, loss 0.349403, acc 0.8125
2018-08-05T21:09:05.756078: step 6345, loss 0.301371, acc 0.8125
2018-08-05T21:09:05.853140: step 6346, loss 0.211396, acc 0.90625
2018-08-05T21:09:

2018-08-05T21:09:24.066202: step 6456, loss 0.340635, acc 0.796875
2018-08-05T21:09:24.162599: step 6457, loss 0.243673, acc 0.890625
2018-08-05T21:09:24.254177: step 6458, loss 0.19005, acc 0.921875
2018-08-05T21:09:24.345774: step 6459, loss 0.344021, acc 0.796875
2018-08-05T21:09:24.440097: step 6460, loss 0.202714, acc 0.921875
2018-08-05T21:09:24.534509: step 6461, loss 0.368664, acc 0.859375
2018-08-05T21:09:24.631381: step 6462, loss 0.386414, acc 0.8125
2018-08-05T21:09:24.725597: step 6463, loss 0.306466, acc 0.84375
2018-08-05T21:09:24.817176: step 6464, loss 0.259161, acc 0.890625
2018-08-05T21:09:24.905805: step 6465, loss 0.425282, acc 0.84375
2018-08-05T21:09:24.992902: step 6466, loss 0.347507, acc 0.84375
2018-08-05T21:09:25.082793: step 6467, loss 0.274729, acc 0.90625
2018-08-05T21:09:25.173808: step 6468, loss 0.362863, acc 0.828125
2018-08-05T21:09:25.258682: step 6469, loss 0.29162, acc 0.890625
2018-08-05T21:09:25.340844: step 6470, loss 0.336664, acc 0.859375
201

KeyboardInterrupt: 

In [15]:
# run this to clear tf flags without re-starting kernel

def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()
    keys_list = [keys for keys in flags_dict]
    for keys in keys_list:
        FLAGS.__delattr__(keys)
        
del_all_flags(tf.flags.FLAGS)

## Testing the Shen model results

In [51]:
# loading model data 
thedon_teen_style_tsf = pd.read_csv("../../results/thedonald/reddit_thedonald.test.0.tsf", delimiter="\n", quoting=3, header = None, names = ['sentence_text'])
teen_thedon_style_tsf = pd.read_csv("../../results/thedonald/reddit_thedonald.test.1.tsf", delimiter="\n", quoting=3, header = None, names = ['sentence_text'])

thedon_teen_style_tsf['group']='teenagers'
teen_thedon_style_tsf['group']='thedonald'

#combine dem/rep data together for training/testing
#shuffle it for random order
teen_thedon_style_tsf_smpl = teen_thedon_style_tsf.sample(n=58403)

all_transf = thedon_teen_style_tsf.append(teen_thedon_style_tsf_smpl)

all_transf=all_transf.sample(frac=1)

print(all_transf.head())

                                sentence_text      group
21355            it was created for the drama  teenagers
21048              the <unk> line that s good  teenagers
2022                        edit i m not once  teenagers
20578  you can just get out of full of course  thedonald
55430                                     k b  teenagers


In [52]:
print(all_transf.shape)

(116806, 2)


In [53]:
test_x = all_transf['sentence_text']
test_y = all_transf['group']

In [54]:
# tokenize data in comment body

tokenizer = TweetTokenizer()

test_x_tokens = []
    
for x in test_x:
    test_x_tokens.append(tokenizer.tokenize(x))

In [55]:
test_x_ids = []

for x in test_x_tokens:
    test_x_ids.append(vocab.words_to_ids(x))

test_x_fdict = []
    
for x in test_x_ids:
    test_x_fdict.append(collections.Counter(x))

test_x_vector = []

num_features = vocab.size
print('creating id lists...')

for x in test_x_fdict:
    test_x_vector.append([x.get(i, 0) for i in range(num_features)])

# use w266 common utils to convert id lists to sparse bow matrix
print('starting id lists to sparse bow conversion...')
test_x_sparse_bow = utils.id_lists_to_sparse_bow(test_x_fdict, vocab.size)

# use Multinomial Naive Bayes to classify transfer sentences
print('generating predictions on transferred data...')
y_pred = nb.predict(test_x_sparse_bow)

acc = accuracy_score(test_y, y_pred)
print("Accuracy on test set: {:.02%}".format(acc))
classrep = classification_report(test_y, y_pred)
print(classrep)

creating id lists...
starting id lists to sparse bow conversion...
generating predictions on transferred data...
Accuracy on test set: 52.11%
             precision    recall  f1-score   support

  teenagers       0.52      0.70      0.59     58403
  thedonald       0.53      0.34      0.42     58403

avg / total       0.52      0.52      0.51    116806



In [ ]:
df = pd.DataFrame({'label':y_pred, 'count':1})

In [ ]:
df.loc[df['label'] == 'thedonald', 'count'] = 0

In [48]:
df['count'].sum()

98980